# `DSML Workshop 12` - Introduction to Neural Networks

In this workshop we provide a very short introduction to neural networks in Python. This is very far from a comprehensive coverage of the topic but can provide a quick start for those who wish to learn more about the topic in their own time. We will cover a classification taks using `Keras` as our python package of choice. If you want to try and implement a NN from scratch there are several good online tutorials that can help you do so (see [here](https://towardsdatascience.com/how-to-build-your-own-neural-network-from-scratch-in-python-68998a08e4f6) for example).

## Biological inspiration
The (for our purpose) smallest stand-alone element in the human brain is the neuron. Its understanding and computational recreation build the foundation for ANNs. A simplified image of a "real" neuron can be seen below.

![](bio_neuron.png)

Dendrites are connecting to the axons (or "outputs") of other neurons, for instance nerves in the sensory system or other processing neurons. In the nucleus, these input signals are aggregated and forwarded through the axon. The axon terminals then connect to further neurons to build the neural network. The connection between axon terminal and dendrite is what we are calling a synapse. In the human brain, there are billions of neurons and $10^{14} - 10^{15}$ synapses. If each synapse (or more precisely, its connection strength) would be represented by 8 bits or one byte, just storing these numbers would take 1000 TB already. 

## Computational implementation
To re-create neural networks artificially, neurons have to be defined. The common mathematical model used for this purpose is depicted below.

![](math_neuron.jpeg)

From a certain number of input synapses $x_i$, signals come in with a weight factor of $w_i$. This represents the strength of the synapse. In the _nucleus_ these weighted inputs are aggregated and a bias is added (the bias is not shown in every model, but it does make the neural network more generalizable). After adding of the weighted inputs and the bias, everything is fed into a (non-linear) activation function. The output is then either fed forward to further neurons or is the output of your neural network. If there is only one neuron that takes direct inputs and whose output is your interest, the model is called a single-layer perceptron. Many of these neurons can create almost arbitrary logical connections and functions, making ANNs very powerful. In this case, we are talking about a multi-layer perceptron (MLP) model. 

![](mlp-network.png)

### Activation Function
The activation function is (to some degree) the heart of the neural network. Without a non-linear activation function, all hidden layers do not add any value, but are instead a complicated way to represent a linear model. Only with a non-linear activation function ANNs can re-create non-linear hypothesis functions. In the beginning of research on ANNs in the scope of AI, typically a unit-step was used as activation function. The unit step is $0$ for inputs smaller than $0$ and $1$ otherwise. The idea behing this is to recreate the behavior of a biological neuron that _fires_ if a certain threshold of inputs is exceeded. Today, other activation functions are more typically used. This is linked to better mathematical qualities in terms of learning behavior and convergence. Some of the most popular activation functions are:

Sigmoid: $\sigma(z) = \frac{1}{1+exp(-z)}$

Hyperbolic tangent: $\sigma(z) = \frac{2}{1+exp(-2z)} -1 $

ReLU (Rectified Linear Unit): $\sigma(z) = z\quad  for\ z>0,\ 0\ otherwise$

### Learning
As learning of ANNs is a non-trivial mathematical task, we are only aiming for an intuitive understanding here. Let's have a look at our complete MLP first.

The general learning task consists of two steps, which are repeated until the algorithm converges:
1. __Feedforward: Calculating the predicted output ŷ and the associated loss__. At first, we randomly assign values for the weights (and the biases). Based on the input features, the output value is calculated.
2. __Backpropagation: Updating the weights W and biases b__. If the output value and the target value differ, the weights and biases are updated. To do this, it is calculated how much each weight and bias contributes to the error. Proportionally to this, they are then corrected (scaled with a small learning factor). In this sense, the updating rule has some similarity to gradient descent, only that is is propagated through the entire network, which is why this algorithm is called backpropagation.

The training routine for a simple 2-layered MPL is shown in the below figure:

![](training.png)

### Hyperparameters
The main hyperparameters of an MLP are: 

1. Number of hidden layers
1. Number of nodes
4. Activation function - could be understood as a hyperparameter, but that is typically not done

The more layers and nodes there are (and the denser the network is, i.e. the more edges have a non-zero weight) the harder it gets to learn the model. That's the reason why bigger ANNs are normally not trained on a local computer anymore, but on specialized computers. Furthermore, there are additional libraries for python to improve the efficiency of ANNs, e.g. TensorFlow or Keras, which we take a first look at in today's tutorial.


# `Keras`

`Keras` is one of the most popular Deep Learning libraries. `Tensorflow` and `Theano` are the most used numerical platforms in Python to build Deep Learning algorithms but they can be quite complex and difficult to use.

Keras, by contrast, is easy to use and is capable of running on top of TensorFlow, Microsoft Cognitive Toolkit, Theano, and MXNet. The full documentation of the Keras API can be found [here](https://keras.io).

Note that `scikit learn` also features an MLP implementation (see [here](https://scikit-learn.org/stable/modules/neural_networks_supervised.html)). Yet, `Keras` has advanced to be one of the most popular frameworks used in practice, which is why we focus on it in this short tutorial.

`Keras` sits on top of `TensorFlow`. Therefore, we fist need to intall the latter library. To do so execute the following command:

`conda install pip`

`pip install tensorflow`

When you are done use the following command via the command line to install `Keras`.

`conda install -c conda-forge keras`.

## Neural networks for classification in `keras`

To stay with our example, we will build a NN that predicts the class of a breast cancer sample by categorizing it as either malignant or benign.

In [1]:
# import standard libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

np.set_printoptions(suppress=True) # suppress scientific notation

# supress versioning warnings of keras
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# import keras libraries
# from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout

2023-06-30 18:10:33.035668: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


__Data Preparation__

In [3]:
# load data
cancer_df = pd.read_csv("breast_cancer.csv", index_col = "id")
cancer_df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
id,,,,,,,,,,,,,,,,,,,,,
842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [4]:
# define x and Y
X = cancer_df.iloc[:,1:31] # include full feature vector
y = cancer_df["diagnosis"]


# encode categorical target verctor
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [5]:
# examine X
X

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
842302,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
842517,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
84300903,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
84348301,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
84358402,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926424,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
926682,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
926954,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820


In [6]:
# examine y
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

In [7]:
# conduct train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
# normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

__Initializing and training the ANN__

We start by defining the type of model we want to build. There are two types of models available in Keras: the [Sequential model](https://keras.io/models/sequential/) and the Model class used with [functional API](https://keras.io/models/model/). We select here the Sequential model and simply add the input-, 2 hidden- and the output-layers.

Between them, we are using [dropout](http://www.jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf?utm_content=buffer79b43&utm_medium=social&utm_source=twitter.com&utm_campaign=buffer) to prevent overfitting (dropout rate should be between 20% and 50% as a rule of thumb).

At every layer, we use “Dense” which means that the nodes are fully connected (i.e., there are connection to each node in the next layer).

The input-layer takes 30 inputs (because our feature vector includes 30 features) as input and outputs it with a shape of 15, which is the number of nodes in the first hidden layer that we define.

In [9]:
# initialize the ANN
classifier = Sequential()

We need to pass the following parameters:

- input_shape - number of columns of the dataset (only for input layer)

- units - number of neurons and dimensionality of outputs to be fed to the next layer, if any

- activation - activation function - we will use ReLU here


In [10]:
# adding the input layer and the first hidden layer (with 30 nodes and 15 nodes respectively)
classifier.add(Dense(input_shape = (30,), 
                     units=15,
                     activation='relu'))

# adding dropout to prevent overfitting
classifier.add(Dropout(rate=0.1))

We add an additional second layer, also with 15 nodes.

In [11]:
# adding the second hidden layer
classifier.add(Dense(units= 15,
                     activation='relu'))

# adding dropout to prevent overfitting
classifier.add(Dropout(rate=0.1))

Finally we add the output layer. Since we perform a binary classification, a single output node suffices. We use a sigmoidal activation function for this last node which is often used when dealing with binary classfication problems.

In [12]:
# adding the output layer
classifier.add(Dense(units = 1, 
                     activation='sigmoid'))

First we compile the model to configure it for training. We add the following parameters:
- `optimizer`: Here we use the adam optimizer, an optimizer with higher performance in many cases than stochastic gradient descent (SGD). See [here](https://keras.io/optimizers/) for a list of all optimizers implemented in `Keras`.
- `loss`: specifies the loss to be minimized. In this example we use binary cross-entropy, a common loss for binary classification tasks. See [here](https://keras.io/losses/) for an overview of available losses in `Keras`. 
- `metrics`:  metric function is similar to a loss function, except that the results from evaluating a metric are not used when training the model and merely function as indicator of model performance to the data scientist. An overview of available metrics can be found [here](https://keras.io/metrics/).

In [13]:
# compiling the ANN
classifier.compile(optimizer="adam", 
              loss="binary_crossentropy",
              metrics=["accuracy"])

In [14]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                465       
                                                                 
 dropout (Dropout)           (None, 15)                0         
                                                                 
 dense_1 (Dense)             (None, 15)                240       
                                                                 
 dropout_1 (Dropout)         (None, 15)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 16        
                                                                 
Total params: 721
Trainable params: 721
Non-trainable params: 0
_________________________________________________________________


**Exercise**: Can you sketch the network? And can you calculate the number of parameters per layer?

We are now able to train our model. We do this with a batch_size of 50 and for 50 epochs.

- `batch_size` defines the number of samples that will be propagated through the network 
- `epoch` defines the number of iteration over the entire training data

In general, a larger batch-size results in faster training, but does not always converge fast. A smaller batch-size is slower in training but it can converge faster. This is definitely problem-dependent and you need to try out a few different values (the standard batch-size is 32). The same goes for the number of epochs.

In [15]:
# fitting the ANN to the training set
classifier.fit(X_train, y_train, batch_size=50, epochs=50)

Epoch 1/50
8/8 [==============================] - 3s 18ms/step - loss: 0.6354 - accuracy: 0.5503
Epoch 2/50
8/8 [==============================] - 0s 7ms/step - loss: 0.5463 - accuracy: 0.7613
Epoch 3/50
8/8 [==============================] - 0s 17ms/step - loss: 0.4963 - accuracy: 0.8342
Epoch 4/50
8/8 [==============================] - 0s 5ms/step - loss: 0.4473 - accuracy: 0.8995
Epoch 5/50
8/8 [==============================] - 0s 7ms/step - loss: 0.3973 - accuracy: 0.9296
Epoch 6/50
8/8 [==============================] - 0s 13ms/step - loss: 0.3705 - accuracy: 0.9196
Epoch 7/50
8/8 [==============================] - 0s 7ms/step - loss: 0.3228 - accuracy: 0.9397
Epoch 8/50
8/8 [==============================] - 0s 13ms/step - loss: 0.2890 - accuracy: 0.9397
Epoch 9/50
8/8 [==============================] - 0s 11ms/step - loss: 0.2745 - accuracy: 0.9372
Epoch 10/50
8/8 [==============================] - 0s 6ms/step - loss: 0.2470 - accuracy: 0.9372
Epoch 11/50
8/8 [=================

In [16]:
# report classification performance on test set
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

confusion_matrix = confusion_matrix(y_test, classifier.predict(X_test).round(decimals=0).astype(int))
accuracy_score = accuracy_score(y_test, classifier.predict(X_test).round(decimals=0).astype(int))
precision_score = precision_score(y_test, classifier.predict(X_test).round(decimals=0).astype(int))

print("Confusion Matrix")
print(confusion_matrix)
print()
print("Accuracy")
print(accuracy_score.round(decimals=4))
print()
print("Precision")
print(precision_score.round(decimals=4))

6/6 [==============================] - 0s 15ms/step
Confusion Matrix
[[108   0]
 [  2  61]]

Accuracy
0.9883

Precision
1.0


Note that this predictive performance is higher than anything we have achieved with traditional models in previous workshops thus far!

## Neural networks for regression in `Keras`

Neural networks can also be trained for regression tasks. The logic is exactly the same, yet some of the parameters, such as loss, metrics, input and ouput as well as typical activation functions might have to be adapted to the specific case. We will not cover ANN regression in this tutorial, which is simply meant as an introduction to the topic. There are a range of very good tutorials online, which we encourage you to take a look at (for example [here](https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/)).

**Exercise:** Starting with a very simple architecture and trying out different values for the key hyperparameters (e.g. via grid search), design, train, validate and test an ANN for the electricity demand dataset which we have worked with throughout this course. What predictive performance do you achieve on the test set? How do you avoid overfitting?

Here are performance metrics of some of our previous models (using only high temperature as raw input to predict peak demand):

**KNN regression**
- MAE: 0.128 GW
- RMSE: 0.159 GW
- R2: 0.755

**Tree-based regression**
- MAE: 0.134 GW
- RMSE: 0.169 GW
- R2: 0.725

Can you design a Neural network that performs better?

In [19]:
# load data
pittsburgh_df = pd.read_csv("Pittsburgh_load_data.csv")

pittsburgh_df.head()

,Date,AVG,MAX,MIN,Total,High_temp,Avg_temp
0,01.01.2013,1.598524,1.859947,0.001599,38.368031,0.0,-1.68
1,02.01.2013,1.809347,2.054215,0.001809,43.428194,-3.9,-6.58
2,03.01.2013,1.832822,2.049550,0.001833,43.991607,0.6,-6.12
3,04.01.2013,1.812699,2.008168,0.001813,43.508609,0.0,-1.95
4,05.01.2013,1.662036,1.838251,0.001662,39.892360,1.7,-1.47


In [20]:
# define x and y
X = pittsburgh_df["High_temp"].values.reshape(-1,1)
y = pittsburgh_df["MAX"]

In [21]:
# conduct train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [22]:
# normalize the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
def nn_regression(hidden_layers = 2, activation_hidden="relu", units_hidden=15, dropout=0.1, batch_size=50, epochs=50):
    # initialize the ANN
    regression = Sequential()

    # adding the input layer and the first hidden layer (with 1 node and 15 nodes respectively)
    regression.add(Dense(input_shape = (1,), 
                        units=units_hidden,
                        activation=activation_hidden))

    # adding dropout to prevent overfitting
    regression.add(Dropout(rate=dropout))

    for i in range(hidden_layers-1):
        # adding hidden layer
        regression.add(Dense(units=units_hidden,
                            activation=activation_hidden))

        # adding dropout to prevent overfitting
        regression.add(Dropout(rate=dropout))

    # adding the output layer
    regression.add(Dense(units = 1)) # unlike classification, no activation function necessary in regression case

    # compiling the ANN
    regression.compile(optimizer="adam", 
                loss="mean_squared_error",
                metrics=["mean_squared_error"])
    
    print(regression.summary())

    # fitting the ANN to the training set
    regression.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)

    # compute test set performance measures
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    print("Test set performance:")

    print("MAE:",mean_absolute_error(y_test, regression.predict(X_test)), "GW")
    print("RMSE:",(mean_squared_error(y_test, regression.predict(X_test)))**(0.5), "GW")  
    print("R2:",r2_score(y_test, regression.predict(X_test)))

In [24]:
nn_regression()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 15)                30        
                                                                 
 dropout_2 (Dropout)         (None, 15)                0         
                                                                 
 dense_4 (Dense)             (None, 15)                240       
                                                                 
 dropout_3 (Dropout)         (None, 15)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 16        
                                                                 
Total params: 286
Trainable params: 286
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
26/26 [==============================] -

---